Q1.1 - Use of two rigid body transformation

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import utils as ut

I = np.eye(3)
Z = np.zeros((3,3))

#Position vectors of points
l_wx = np.array([[1], [-0.1], [0.4]]) #in frame B1
l_yz = np.array([[1], [0], [0.2]]) #in frame B2

#Spatial velocities of w with respect to I in frame B1
V_Iw = np.array([1.2,0.7,0,0.3,0,2.1]).reshape((6,1))

#Quarternions representing orientations
q_IB1 = np.array([1/(np.sqrt(3)*2),1/(np.sqrt(3)*2), 1/(np.sqrt(3)*2), np.sqrt(3)/2]).reshape(4,1) #B1 to I
q_B1B2 = q_IB1 #B2 to B1 (same rotation as B1 to I)

#Hinge map in non-rigid case
H = np.block([np.eye(3), np.zeros((3,3))])

#Rotation matrices
R_IB1 = ut.rotfromquat(q_IB1)
R_B1B2 = ut.rotfromquat(q_B1B2)

#1.1.1 Single transformation
l_wz = R_IB1 @ (l_wx + R_B1B2 @ l_yz) #Vector from w to z in inertial frame
phi_wz = ut.rigidbodytransform(l_wz) #in intertial frame

V_Iz = phi_wz.T @ np.block([[R_IB1, Z],[Z, R_IB1]]) @ V_Iw #Spatial velocity of point z with respect to I in inertial frame

print(V_Iz)

#1.1.2 Chaining
theta_dot = np.array([3.2, 0, 1.2]).reshape((3,1)) #Generalized hinge velocities

phi_wx = ut.rigidbodytransform(l_wx) #in frame B1

#calculating spatial velocity at y - so first tranform from w to x, then adding the hinge contribution

[[ 0.56666667]
 [ 1.26666667]
 [ 0.06666667]
 [ 1.33333333]
 [-0.34666667]
 [ 0.65333333]]
